# Packages

[earth engnie](https://earthengine.google.com/) | [geemap](https://geemap.org/) | [osmnx](https://osmnx.readthedocs.io/en/stable/) | [geopandas](https://geopandas.org/en/stable/)


In [31]:
!pip install -q osmnx;

import ee
import geemap
import osmnx as ox
import geopandas as gpd
from google.colab import output, userdata
import warnings; warnings.filterwarnings('ignore');

Setup with [Initialize GEE]( #https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=hufSdD1Q-7Y8ztJq-C0aM3oxLKP3Z6lsKaY9G95WlLc&tc=NpxgbkWSWr426fIx_drps7CDYFnF_ECs3De8hmdDMOY&cc=06Hry_2zntT1furCVleTfZNz5P4uozVHzqGa7gwe-Jc
)

In [32]:
try:
  ee.Initialize();
except:
  project_name = userdata.get('my_project_id') # --> Your Google Cloud project ID
  ee.Authenticate()
  ee.Initialize(project = project_name);
  del project_name

# Configutarion Variables

- [Golan Heights & Galile coded values by FAO](https://data.apps.fao.org/?source=https%3A%2F%2Fdata.apps.fao.org%2Fmap%2Fgsrv%2Fgsrv1%2Fgaul%2Fwms&layers=g2015_2014_1&type=wms&name=g2015_2014_1&lang=en&share=f-807f9ba5-0f16-4d29-8d1b-e772f70288b8)

In [33]:
before_start: str = '2024-05-15'
before_end: str   = '2024-05-31'

after_start: str = '2024-06-01'
after_end: str   = '2024-06-20'

Galile_GAUL:int  = 1613
Golan_GAUL: int  = 2833

BiriyaCoords: list[float] = [32.99, 35.51]
SefadCoords: list[float] = [32.9644, 35.4952]
MeronCoords: list[float] = [32.9846, 35.4412]
HatzorHaglilit: list[float] = [32.9826, 35.25449]

SefadCoords_rev: list[float] = [35.4952, 32.9644]
MeronCoords_rev: list[float] = [35.4412, 32.9846]
HatzorHaglilit_rev: list[float] = [35.25449, 32.9826]

# Functions

In [34]:
def FeatureCentroid(feature: ee.Feature)-> list[float, float]:
  """
  Returns the center point coordinates of an ee.Feature object as a list of X and Y.
  """
  center = feature.geometry().centroid().coordinates().getInfo()
  center[0], center[1] = center[1], center[0]

  return center


def compute_NBR(S2Image: ee.Image) -> ee.Image:
  """
  Computes the Normalized Burn Ratio (NBR) from a given Sentinel-2 image.

  The NBR is a simple index that is sensitive to the presence of burned areas.
  It is calculated as the difference between the near-infrared (B8) and shortwave infrared (B12) bands,
  normalized by their sum.

  Parameters:
  S2Image (ee.Image): A Sentinel-2 image from Google Earth Engine.

  Returns:
  ee.Image: A single-band image containing the NBR values.
  """

  NBR_expression: str = '(B8-B12)/(B8+B12)'

  NBR_image: ee.Image = S2Image.expression(NBR_expression,
                                           {'B8' : S2Image.select('B8'),
                                            'B12': S2Image.select('B12')})

  NBR_image = NBR_image.rename('NBR')

  return NBR_image


def compute_delta(before_image: ee.Image, after_image: ee.Image) -> ee.Image:
  """
  Computes the difference between two input images, which can be used to detect changes in land cover.

  Parameters:
  before_image (ee.Image): The first input image, representing the state before a change event.
  after_image (ee.Image): The second input image, representing the state after a change event.

  Returns:
  ee.Image: A single-band image containing the difference between the two input images.
  """

  delta = before_image.subtract(after_image)
  delta = delta.rename('Delta')

  return delta

# Lications & Area of Interest

* An area of intereset (aoi) defined by merging the borderds of the Golag heights and Galile districts.

* The water bodies is extructed from OpenStreetMap data to aplly a mask that exclude water pixels from aoi.



In [35]:
Galile = ee.FeatureCollection('FAO/GAUL/2015/level1').filterMetadata('ADM1_CODE','equals',Galile_GAUL) # --> Galile district of Israel
Golan = ee.FeatureCollection('FAO/GAUL/2015/level1').filterMetadata('ADM1_CODE','equals',Golan_GAUL)   # --> Golan Heights district of Israel
aoi = Galile.merge(Golan).union()

water_bodies = ox.features_from_place('Israel', {'natural': ['water']}).reset_index(drop=True)[['geometry']]
water_bodies = water_bodies[~water_bodies.geom_type.isin(['Point', 'LineString'])]
water_bodies = water_bodies.dissolve();
ee_water_bodies = geemap.gdf_to_ee(water_bodies)

aoi = geemap.ee_to_gdf(aoi).dissolve();
aoi = aoi.difference(water_bodies.geometry, align=False);
aoi = gpd.GeoDataFrame(aoi, index=[0]).rename(columns = {0:'geometry'}).set_geometry('geometry').set_crs(epsg=4326);

aoi = geemap.gdf_to_ee(aoi)
aoi_center: list[float] = FeatureCentroid(aoi)


Sefad_ee = ee.Feature(ee.Geometry.Point(SefadCoords_rev), {'Name': 'Sefad'})
Meron_ee = ee.Feature(ee.Geometry.Point(MeronCoords_rev), {'Name': 'Meron'})
HatzorHaglilit_ee = ee.Feature(ee.Geometry.Point(HatzorHaglilit_rev), {'Name': 'Hatzor HaGlilit'})

orientation_points = ee.FeatureCollection([Sefad_ee, Meron_ee, HatzorHaglilit_ee])

# Collect images


For each specified date range (both before and after a given event), the following steps were performed:

1. A true color image was retrieved.
2. An NBR (Normalized Burn Ratio) image was computed from the true color image.
3. The difference between the NBR images (delta NBR) for the before and after date ranges was calculated.
4. The delta NBR values were reclassified into a scale from 1 to 5.

In [36]:
S2_before = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
              .filterBounds(aoi)\
              .filterDate(before_start, before_end)\
              .mosaic()\
              .clip(aoi)\
              .divide(100)


S2_after = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
             .filterBounds(aoi)\
             .filterDate(after_start, after_end)\
             .mosaic()\
             .clip(aoi)\
             .divide(100)


NBR_before = compute_NBR(S2_before)
NBR_after = compute_NBR(S2_after)


dNBR = compute_delta(NBR_before, NBR_after).rename('dNBR')
NBR_Sevirity = ee.Image(0).where(dNBR.lt(-0.8),                    1)\
                          .where(dNBR.gte(-0.8).And(dNBR.lt(0.4)), 2)\
                          .where(dNBR.gte(0.4).And(dNBR.lt(0.6)),  3)\
                          .where(dNBR.gte(0.6).And(dNBR.lt(0.85)), 4)\
                          .where(dNBR.gte(0.85),                   5)\
                          .rename('Sevirity')\
                          .clip(aoi)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Defining visualizations and legends dictionaries

* [CSS Color codes](https://www.quackit.com/css/css_color_codes.cfm)

In [37]:
true_color_vis = {'min': 0,   'max': 100, 'bands': ['B4', 'B3', 'B2']}

NBR_vis        = {'min': -1,  'max': 1,  'palette': ['black', 'DarkRed', 'orange', 'green', 'LightGreen'], 'bands': ['NBR']}
dNBR_vis       = {'min': -2,  'max': 2,  'palette': ['LightGreen', 'SpringGreen', 'green',  'yellow', 'orange', 'DarkOrange', 'red', 'DarkRed', 'black'], 'bands': ['dNBR']}
Sevirity_vis   = {'min':  1,  'max': 5,  'palette': ['white',  'LightGrey', 'orange', 'red', 'DarkRed'], 'bands': ['Sevirity']}

water_bodies_vis = {'color': 'Aqua'}


NBR_legend = {'Burned Forest':             '#000000', #black
              'Severely Damaged Forest':   '#8B0000', #DarkRed
              'Moderately Damaged Forest': '#FFA500', #orange
              'Recovering Forest':         '#008000', #green
              'Healthy Forest':            '#90EE90'} #LightGreen


Sevirity_legend = {1: '#FFFFFF', #white
                   2: '#D3D3D3', #LightGrey
                   3: '#FFA500', #orange
                   4: '#FF0000', #red
                   5: '#8B0000'} #DarkRed

# Visualization

In [38]:
Map = geemap.Map(center = BiriyaCoords, zoom = 14, add_google_map=False)


Map.addLayer(ee_object = S2_before, vis_params = true_color_vis, name = 'Before', opacity = 1)
Map.addLayer(ee_object = S2_after, vis_params = true_color_vis, name = 'After', opacity = 1)

Map.addLayer(ee_object = NBR_before, vis_params = NBR_vis, name = 'NBR before', opacity = 1)
Map.addLayer(ee_object = NBR_after, vis_params = NBR_vis, name = 'NBR after', opacity = 1)
Map.add_legend(title = 'NBR', legend_dict = NBR_legend, position = "bottomleft")

Map.addLayer(ee_object = NBR_Sevirity, vis_params = Sevirity_vis, name = 'Burn Sevirity', opacity = 1)
Map.add_legend(title = 'Sevirity', legend_dict = Sevirity_legend, position = "bottomleft")

Map.add_gdf(water_bodies, layer_name= 'Water bodies', style= water_bodies_vis, zoom_to_layer=False, fill_colors=['Aqua'])

Map.addLayer(ee_object = orientation_points, vis_params = {'color': 'DeepPink'}, name = 'Local Communities', opacity = 0)
Map.add_labels(orientation_points, column = "Name", font_size = "14pt", font_color = "DeepPink", font_family = "Segoe UI", font_weight = "bold")

Map

Map(center=[32.99, 35.51], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [39]:
output.enable_custom_widget_manager()

LinkMaps = geemap.linked_maps(rows = 2, cols = 2,
                              height = "400px", center = BiriyaCoords, zoom = 14,
                              ee_objects = [S2_before, S2_after, NBR_before, NBR_after],
                              vis_params = [true_color_vis, true_color_vis, NBR_vis, NBR_vis],
                              labels     = ['Before', 'After', 'Before', 'After'])


LinkMaps

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

# Export the reulting images to Google Drive folder
Consider downgrading the scale parameter if resulting image is to large or clip the extent of the image

In [ ]:
scale = 20
FileName: str = 'FileName' # --> Name your tif file
output_tiff_path: str = fr'/content/drive/MyDrive/{FileName}.tif'

geemap.ee_export_image(ee_object = NBR_Sevirity, filename = output_tiff_path, scale= scale, region= aoi.geometry(), file_per_band= True);